In [6]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time

In [10]:
driver = uc.Chrome()

In [14]:
driver.get('https://www.gelbeseiten.de/suche/steuerberatung/essen?umkreis=50000')
time.sleep(3)

In [13]:
driver.find_element(By.XPATH, '//*[@id="cmpwelcomebtnyes"]/a').click()

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (222, 783)
  (Session info: chrome=127.0.6533.72)
Stacktrace:
	GetHandleVerifier [0x0082F653+11395]
	(No symbol) [0x0080B7A4]
	(No symbol) [0x00751A43]
	(No symbol) [0x0079B5C7]
	(No symbol) [0x007999D9]
	(No symbol) [0x007978CB]
	(No symbol) [0x00796EAF]
	(No symbol) [0x0078BFAF]
	(No symbol) [0x007B719C]
	(No symbol) [0x0078BA45]
	(No symbol) [0x007B7434]
	(No symbol) [0x007D03DC]
	(No symbol) [0x007B6F36]
	(No symbol) [0x0078A276]
	(No symbol) [0x0078AECD]
	GetHandleVerifier [0x00932403+1071667]
	GetHandleVerifier [0x0093C9C4+1114100]
	GetHandleVerifier [0x009354B2+1084130]
	GetHandleVerifier [0x0085013B+145259]
	(No symbol) [0x00811693]
	(No symbol) [0x0080EFB8]
	(No symbol) [0x0080F157]
	(No symbol) [0x0080842E]
	BaseThreadInitThunk [0x766AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x776F80CE+286]
	RtlGetAppContainerNamedObjectPath [0x776F809E+238]


In [5]:
total = 3933
loadmorebutton = driver.find_element(By.XPATH, '//a[@id="mod-LoadMore--button"]')
profiles =driver.find_elements(By.XPATH, '//div[@id="teilnehmer_block"]/div/article')

while len(profiles)!= total:
    try:
        driver.execute_script("arguments[0].scrollIntoView(false);", loadmorebutton)
        loadmorebutton.click()
        count = len(profiles)
        print(count)
        time.sleep(3)
    except:
        loadmore = False


50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
